In [1]:
import os
%pwd

'/mnt/cb03386d-9344-47b1-82f9-868fbb64b4ae/python_projects/Cell_Segmentation_YOLO-v8/research'

In [2]:
os.chdir('../')
%pwd

'/mnt/cb03386d-9344-47b1-82f9-868fbb64b4ae/python_projects/Cell_Segmentation_YOLO-v8'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    model_names: list
    dataset_yaml: Path
    model_params: dict

In [4]:
from src.cellseg.constant import *
from src.cellseg.utils.main_utils import create_directories, read_yaml

class ConfigurationManager:
    def __init__(
        self,
        config_file_path = CONFIG_FILE_PATH,
        params_file_path = PARAMS_FILE_PATH,
        schema_file_path = SCHEMA_FILE_PATH
    ):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)
        
        create_directories([self.config.artifacts_root])
    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.model
        
        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            model_names=config.model_names,
            dataset_yaml=config.dataset_yaml,
            model_params=params
        )
        
        return model_trainer_config

In [9]:
from ultralytics import YOLO
from src.cellseg.utils.main_utils import create_directories

class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config
    
    def train(self):
        for model_name in self.config.model_names:
            model_folder = os.path.join(self.config.root_dir, str.split(model_name, '.')[0])
            project_path = os.path.join(model_folder, 'results')
            
            create_directories([project_path])
            
            model = YOLO(os.path.join(model_folder, model_name))
            
            params = self.config.model_params
            params['data'] = self.config.dataset_yaml
            params['project'] = project_path

            model.train(**params)

In [10]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    results = model_trainer.train()

except Exception as e:
    raise e

[2024-11-13 19:28:13,218: INFO: main_utils: created directory at: artifacts]
[2024-11-13 19:28:13,219: INFO: main_utils: created directory at: artifacts/model_trainer/yolo11n-seg/results]


100%|██████████| 5.90M/5.90M [00:00<00:00, 15.8MB/s]


Ultralytics 8.3.29 🚀 Python-3.10.12 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce GTX 1060 6GB, 6065MiB)
engine/trainer: task=segment, mode=train, model=artifacts/model_trainer/yolo11n-seg/yolo11n-seg.pt, data=artifacts/data_transformation/dataset.yaml, epochs=200, time=None, patience=0, batch=32, imgsz=256, save=True, save_period=-1, cache=False, device=0, workers=4, project=artifacts/model_trainer/yolo11n-seg/results, name=200_epochs_, exist_ok=False, pretrained=True, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=Fals

[W1113 19:28:15.530939643 NNPACK.cpp:61] Could not initialize NNPACK! Reason: Unsupported hardware.


YOLO11n-seg summary: 355 layers, 2,842,803 parameters, 2,842,787 gradients, 10.4 GFLOPs

Transferred 510/561 items from pretrained weights
Freezing layer 'model.23.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...
AMP: checks passed ✅


train: Scanning /mnt/cb03386d-9344-47b1-82f9-868fbb64b4ae/python_projects/Cell_Segmentation_YOLO-v8/artifacts/data_transformation/train.cache... 2680 images, 19 backgrounds, 0 corrupt: 100%|██████████| 2680/2680 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Scanning /mnt/cb03386d-9344-47b1-82f9-868fbb64b4ae/python_projects/Cell_Segmentation_YOLO-v8/artifacts/data_transformation/validation.cache... 670 images, 5 backgrounds, 0 corrupt: 100%|██████████| 670/670 [00:00<?, ?it/s]


Plotting labels to artifacts/model_trainer/yolo11n-seg/results/200_epochs_/labels.jpg... 

optimizer: Adam(lr=0.01, momentum=0.937) with parameter groups 90 weight(decay=0.0), 101 weight(decay=0.0005), 100 bias(decay=0.0)
Image sizes 256 train, 256 val
Using 4 dataloader workers
Logging results to artifacts/model_trainer/yolo11n-seg/results/200_epochs_
Starting training for 200 epochs...

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


      1/200      1.22G      4.374      6.125      3.614       2.54       1415        256:  11%|█         | 9/84 [00:11<01:37,  1.30s/it]